In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import sys
import datetime
import sqlite3

In [ ]:
# wrap into function
def scrape_restaurant(query):

In [2]:
    # set up driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(options=chrome_options)
    
    # input variables
    # query = "Rosy's Taco Bar"

    description = "Taco bar serving Mexican street food, margaritas & tequila in a colorful, casual setting. Must try the margaritas and the loaded tacos during Happy Hour!"
    
    # connect to database
    db = sqlite3.connect("booking_data.db")
    cursor = db.cursor()
    
    driver.get('https://www.google.com/?hl=en')
    driver.find_element(By.NAME, "q").send_keys(query, Keys.ENTER)
    
    time.sleep(2)

In [3]:
    # get scraping date and time
    scraped_date = datetime.datetime.now().strftime('%Y-%m-%d')
    print(scraped_date)
    scraped_time = datetime.datetime.now().strftime('%H:%M:%S')
    print(scraped_time)

2024-03-17
09:43:30


In [4]:
    # get name
    try:
        restaurant_name = driver.find_element(By.CSS_SELECTOR, f"div[data-attrid='title']").get_attribute("innerText")
        print(restaurant_name)
    except NoSuchElementException:
        restaurant_name = driver.find_element(By.CSS_SELECTOR, f"h2[data-attrid='title']").get_attribute("innerText")
        print(restaurant_name)
    except:
        print("Could not find restaurant name")

Rosy's Taco Bar


In [5]:
    # get address
    try:
        restaurant_address = driver.find_element(By.CSS_SELECTOR,
                                                 f"div[data-attrid='kc:/location/location:address']").get_attribute("innerText")
    except NoSuchElementException:
        restaurant_address = driver.find_element(By.CSS_SELECTOR,
                                                 f"h2[data-attrid='kc:/location/location:address']").get_attribute("innerText")
    
    restaurant_address = restaurant_address.replace("Address: ", "")
    print(restaurant_address)

2220 Walnut St, Philadelphia, PA 19103


In [6]:
    # get review count
    review_count = driver.find_element(By.PARTIAL_LINK_TEXT, "Google reviews").get_attribute("innerText")
    review_count = review_count.split(" ")[0]
    review_count = review_count.replace(",", "")
    print("Review count: " + review_count)        

1007


In [7]:
    # get rating
    rating = driver.find_element(By.CSS_SELECTOR, "span[aria-label*='out of 5']").get_attribute("aria-label") 
    rating = rating.split(" ")[1]
    print("Rating: " + rating)

4.0


In [8]:
    # insert data into database
    row = (restaurant_name, restaurant_address, rating, review_count, description, scraped_date, scraped_time)
    
    try: 
        cursor.execute(
            "INSERT INTO restaurants (restaurant_name, restaurant_address, rating, review_count, description, scraped_date, scraped_time) VALUES (?, ?, ?, ?, ?, ?, ?)", row)
    except sqlite3.IntegrityError as e:
        print("Restaurant already exists in database")
    
    db.commit()
    
    # close connections
    driver.quit()

IntegrityError: UNIQUE constraint failed: restaurants.restaurant_name